In [1]:
import pandas as pd


In [2]:
df=pd.read_csv('/Users/joe/Desktop/sql/data/realtor_data.csv')
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [3]:
df_dropped=df.drop('prev_sold_date',axis=1).copy()
df_dropped.drop_duplicates(inplace=True)


In [4]:
df_dropped.dropna(axis=0,inplace=True)
df_dropped.info()
df_dropped.to_parquet('/Users/joe/Desktop/sql/data/cleaned_data.parquet',engine='fastparquet')


<class 'pandas.core.frame.DataFrame'>
Index: 1353936 entries, 0 to 2226381
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   brokered_by  1353936 non-null  float64
 1   status       1353936 non-null  object 
 2   price        1353936 non-null  float64
 3   bed          1353936 non-null  float64
 4   bath         1353936 non-null  float64
 5   acre_lot     1353936 non-null  float64
 6   street       1353936 non-null  float64
 7   city         1353936 non-null  object 
 8   state        1353936 non-null  object 
 9   zip_code     1353936 non-null  float64
 10  house_size   1353936 non-null  float64
dtypes: float64(8), object(3)
memory usage: 124.0+ MB


In [5]:
# Sample the DataFrame
df_sampled = df_dropped.sample(n=1000000)

# Reset the index, drop the old index, and start the new index from 1
df_sampled = df_sampled.reset_index(drop=True)
df_sampled.index = df_sampled.index + 1

# Save the DataFrame to an Excel file
df_sampled.to_excel('/Users/joe/Desktop/sql/data/cleaned_data.xlsx', engine='xlsxwriter', index=True)


In [6]:
df_cleaned=df_sampled.groupby(['state','city'])['price'].agg('median').reset_index().copy()
df_cleaned

,state,city,price
0,Alabama,Abbeville,215000.0
1,Alabama,Adamsville,150000.0
2,Alabama,Addison,131950.0
3,Alabama,Adger,127450.0
4,Alabama,Akron,465000.0
...,...,...,...
24925,Wyoming,Wheatland,250000.0
24926,Wyoming,Wilson,5925000.0
24927,Wyoming,Worland,229000.0
24928,Wyoming,Wright,274900.0


In [7]:
df_filtered = df_cleaned[df_cleaned['price'] != 1.0]
max_price_city = df_filtered.loc[df_filtered.groupby('state')['price'].idxmax()]
min_price_city = df_filtered.loc[df_filtered.groupby('state')['price'].idxmin()]
df_max_min_prices= pd.concat([max_price_city, min_price_city]).sort_values(by='state').reset_index(drop=True)
df_max_min_prices.to_excel('/Users/joe/Desktop/sql/data/max_min_prices.xlsx',engine='xlsxwriter')